# Nuke button

In [5]:
# Delete all files and folders in the working directory. Use with caution!
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)

# Code

## Git Clone

In [6]:
!git clone -q https://github.com/BinhPQ2/magi_functional.git

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [2]:
!pip install -qr /kaggle/working/magi_functional/requirements_kaggle.txt

## Load MAGI model

In [5]:
from PIL import Image
import numpy as np
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained("ragavsachdeva/magiv2", trust_remote_code=True).cuda().eval()

## Save model to local

In [ ]:
# Save model to directory:
model.save_pretrained("./magi_saved_model_directory/")

In [ ]:
# Load model from directory:
model = AutoModel.from_pretrained("./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

In [ ]:
####################### OR ###########################
# Cache model to directory:
# model_1 = AutoModel.from_pretrained("ragavsachdeva/magiv2", cache_dir="./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

# Code

## Create raw and character/names list

In [7]:
import os
import re
from pathlib import Path

def create_chapter_pages_and_character_bank(manga_folder, character_folder):
    # Create lists for chapter pages and character bank
    chapter_pages = []
    character_bank = {
        "images": [],
        "names": []
    }

#     Iterate through manga images to create chapter_pages
    for image_file in os.listdir(manga_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Extract the page number using regex
            match = re.search(r'p(\d+)', image_file)
            if match:
                page_number = int(match.group(1))  # Convert to integer for sorting
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)
            else:
                page_number = image_file.rsplit(".", 1)[0]
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)

    # Sort chapter pages by page number
    chapter_pages.sort(key=lambda x: x[0])
    chapter_pages = [os.path.join(manga_folder, img[1]) for img in chapter_pages]  # Extract just the filenames after sorting

    # Iterate through character images to create character bank
    for char_image_file in os.listdir(character_folder):
        if char_image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Split the filename to extract character name
            char_name = char_image_file.split('_')[0]  # Get the part before the underscore
            character_bank["images"].append(os.path.join(character_folder, char_image_file))
            character_bank["names"].append(char_name)
    return chapter_pages, character_bank

# Define your folders
# One Piece
# manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/raw_manga")
# character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/character")

# Ruri Dragon
manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw")
character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/character")

# Get chapter pages and character bank
chapter_pages_original, character_bank_original = create_chapter_pages_and_character_bank(manga_folder, character_folder)

chapter_pages_test = chapter_pages_original[:]
character_bank_test = character_bank_original

# Print the results (for debugging)
print("Chapter Pages:")
print(chapter_pages_test)

print("\nCharacter Bank:")
print(character_bank_test)

Chapter Pages:
['/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/00.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/01.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/02.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/03.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/04.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/05.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/06.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/07.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/08.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/09.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/10.jpg', '/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw/11.jpg', 

## Process (OCR → Transcript)

In [11]:
import os
import json

def read_image(path_to_image):
    with open(path_to_image, "rb") as file:
        image = Image.open(file).convert("L").convert("RGB")
        image = np.array(image)
    return image


chapter_pages = [read_image(x) for x in chapter_pages_test]
character_bank = character_bank_test.copy()
character_bank["images"] = [read_image(x) for x in character_bank_test["images"]]

with torch.no_grad():
    per_page_results = model.do_chapter_wide_prediction(chapter_pages, character_bank, use_tqdm=True, do_ocr=True)

print("Continue with next cell")

100%|██████████| 6/6 [00:10<00:00,  1.67s/it]


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/1e2c6516e5524d15a92366d85bf77bda-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/1e2c6516e5524d15a92366d85bf77bda-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 22383 COLUMNS
At line 83100 RHS
At line 105479 BOUNDS
At line 109680 ENDATA
Problem MODEL has 22378 rows, 4200 columns and 48116 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 258.493 - 0.08 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 9194 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8599 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8320 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7889 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tighten

100%|██████████| 14/14 [01:14<00:00,  5.35s/it]


## Save transcript

In [16]:
result_dir = "/kaggle/working/result"
json_output_dir = f"{result_dir}/json_results"
result_image_output_dir = f"{result_dir}/image_results"
transcript_output_dir = f"{result_dir}/transcript.txt"
os.makedirs(json_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  
os.makedirs(result_image_output_dir, exist_ok=True)  # Create the directory if it doesn't exist

transcript = []
for i, (image, page_result) in enumerate(zip(chapter_pages, per_page_results)):
    image_name_ext = os.path.basename(chapter_pages_test[i])
    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    
    model.visualise_single_image_prediction(image, page_result, os.path.join(result_image_output_dir, f"{image_name}.png"))
    # Save page_result to JSON
    json_file_path = os.path.join(json_output_dir, f"{image_name}.json")  # Create full file path
    with open(json_file_path, 'w') as json_file:
        json.dump(page_result, json_file, indent=4)  # Save with pretty printing

    speaker_name = {
        text_idx: page_result["character_names"][char_idx] for text_idx, char_idx in page_result["text_character_associations"]
    }
    for j in range(len(page_result["ocr"])):
        if not page_result["is_essential_text"][j]:
            continue
        name = speaker_name.get(j, "unsure") 
        transcript.append(f"<{name}>: {page_result['ocr'][j]}")
    transcript.append("---------------------------------------------")
with open(transcript_output_dir, "w") as fh:
    for line in transcript:
        fh.write(line + "\n")

print("\n\nDone you WEEEEB!")



Done you WEEEEB!


# Zip file

## Legacy

In [ ]:
# json_output_dir = "/kaggle/working/json_results"
# result_image_output_dir = "/kaggle/working/image_results"

# # zip
# json_output_dir = "/kaggle/working/json_results"
# !zip -rj /kaggle/working/json.zip {json_output_dir}/*.json
# # download
# !cd /kaggle/working/
# display(FileLink('json.zip'))

## Download separate files

### Download json

In [17]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("json_results", f"{json_output_dir}/*.json")

/kaggle/working/json_results.zip

### Download image results

In [18]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", result_image_output_dir)

/kaggle/working/image_results.zip

### Download transcript

In [11]:
import os
os.chdir(result_dir)
display(FileLink('transcript.txt'))

/kaggle/working/transcript.txt

## Download all

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -r {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    

!cp -r result_dir /
download_file("result", result_dir)